### Projeto 2: Ciencia dos dados.

### Alexandre Young , Jean Walper, Pedro de la peña, Sabrina Sampaio ###

### Introdução:

O nosso projeto consiste em propor um teste AB para um site, aplicá-lo e analisar seus resultados.
    
Um teste AB é uma tipo de teste usado para analisar os efeitos de uma caracteristica diferente tem sobre uma determinada tarefa. O utilizamos para estudar os efeitos que a disposição de links teriam na navegação de um site.
  
Como exemplo real utilizamos um site que acreditamos que se beneficiaria dessa mudança, o portfólio de projetos (primeiro projeto de Co-Design) de um dos integrantes do grupo. No modelo original do site (A), o posicionamento dos links para as paginas de projeto parecia causar uma certa confusão sobre quais itens eram links "clicáveis" ao estar abaixo de itens que não eram,para confirmar essa hipótese construímos uma versão modificada (B) onde acreditamos que a disposição dos links fosse mais intuitiva. 

Testamos então a navegabilidade de cada versão do site com o teste específico de encontrar a pagina de um certo projeto fixo no site.

Encontramos 79 candidatos dispostos a nos ajudar no teste, sem conhecer ou ser exposto à outra versão do site e sem nenhuma indicação de que caminho seguir, pedimos que cada candidato encontrasse a página do "Jogo de Computador" no portfolio. Para os 42 que navegaram pela a versão A do site e os 37 pela a versão B procuramos medir apenas o tempo que levaram, em segundos para encontrar a página específica.
  
  

In [2]:
import pandas as pd
import json
import itertools
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
%matplotlib inline

### Leitura do arquivo

In [3]:
arquivo = open("exportNINJA.json")
linhas = arquivo.readlines()
texto = "".join(linhas)
dados_fire = json.loads(texto)

### Análise do atributo logs

 Veja que log, que contém todos os dados, está pesadamente aninhado

In [4]:
for k,v in dados_fire["logs"].items():
    # print(k,v) # habilite para imprimir as chaves de mais alto nível
    for chave, valor in v.items():
        # print(chave, valor) # habilite para imprimir as chaves inferiores
        pass # desabilite quando reabilitar a linha de cima

### Convertendo para um dataframe

O pandas entende listas de dicionários que têm as mesmas chaves. O Firebase nos dá dicionários de dicionários de dicionários. 

Vamos precisar converter. Basta inserir todos os itens que estavam na chave "logs" em uma lista

In [5]:
lista_dicionarios = []

for k,v in dados_fire["logs"].items():
    lista_dicionarios.append(v)

A função `from_records` converte a lista de dicionários em um dataframe

In [6]:
dataframe = pd.DataFrame.from_records(lista_dicionarios)

In [7]:
dataframe

,tempo,tipo,user_id,versao
0,1479337480444,fim,3473480114,A
1,1479388920285,fim,993384729,A
2,1479342231419,inicio,1939264432,A
3,1479401423182,inicio,3839108837,B
4,1479387680103,inicio,1607596965,A
5,1479341841499,inicio,336620398,A
6,1479347570143,inicio,665338756,B
7,1479341948357,fim,570755405,B
8,1479334275930,inicio,1609099169,A
9,1479341997421,inicio,336620398,A


### Convertendo para um DataFrame de delta_t

In [8]:
agrupado = dataframe.groupby(by=["user_id", "versao"])

In [9]:
users = set(dataframe.user_id) # We convert it to a set so that each user appear only once

In [10]:
users = list(users)

In [11]:
resultados = []

for u in users:
    for versao in ["A", "B"]:
        if (u, versao) in agrupado.groups.keys(): # Check if a specific pair user/type of test happened
            grupo1 =  agrupado.get_group((u, versao))
            grupo1 = grupo1.sort_values(by="tempo") # Para evitar que inícios sem fim gerem tempos negativos
            
            if len(grupo1[grupo1.tipo=="fim"])>= 1 and len(grupo1[grupo1.tipo=="inicio"]) >= 1:
                delta_t = grupo1[grupo1.tipo=="fim"].iloc[-1].tempo - grupo1[grupo1.tipo=="inicio"].iloc[0].tempo
                resultados.append({"user_id":u, "delta_t":delta_t, "versao":versao})

In [12]:
grupo1.sort_values(by="tempo")

,tempo,tipo,user_id,versao
164,1479386635776,inicio,8623611,A
176,1479386665449,fim,8623611,A


In [13]:
dataframe_resumo = pd.DataFrame.from_records(resultados)

In [14]:
dataframe_resumo.delta_t/=1000 # Convertendo milissegundos em segundos

In [15]:
dfA= dataframe_resumo[dataframe_resumo.versao == "A"]
dfB= dataframe_resumo[dataframe_resumo.versao == "B"]



print(dfA.shape[0],dfB.shape[0])

dataframe_resumo

42 37


,delta_t,user_id,versao
0,97.183,2867771403,B
1,20.414,3218700317,A
2,108.507,200239137,A
3,187.029,849639971,A
4,204.820,2452834855,A
5,86.461,1996812841,B
6,181.267,1722607667,B
7,18.911,2453186631,A
8,91.973,3215534152,B
9,12.789,105061458,A


# <color='m'> Teste de hipótese! </color>
### Usando teste t, pois nossa amostra < 200

Definimos a hipótese nula como o tempo médio gasto na versão A do site equivalendo ao tempo médio gasto na versão B (H0:μa=μb) e a hipótese alternativa como o valor médio de tempo gasto na versão A sendo maior que o tempo gasto na versão B (H1:μa > μb). O nível de significância inicial foi de 10%.
    
Devido ao fato de não possuirmos muitas amostras, realizamos o teste t-student em vez de um teste z. Os parâmetros do teste estão definidos no código a seguir, sendo T a média padronizada e V o grau de liberdade:

In [16]:
alpha= 0.1 #Nivel de significancia 10%
na= dfA.shape[0]
nb = dfB.shape[0]
D = 0 #Pois adimitimos H0: uma media igual a outra

#Hipótese:
    #H0: μa = μb
    #H1: μa =/= μb
    
μa= dfA.delta_t.mean()
μb= dfB.delta_t.mean()

Sa= dfA.delta_t.std()
Sb= dfB.delta_t.std()

sna = ((Sa**2)/na)
snb = ((Sb**2)/nb)
spa = (sna**2)/(na-1)
spb = (snb**2)/(nb-1)

#LES FORMULAS(3B, magalhães e lima)

T=(D-(μa - μb))/((sna+snb)**(0.5))
v= ((sna + snb)**2)/(spa+spb)
T, v

(-1.8316210162059656, 63.09894073603812)

In [17]:
pvalue = stats.t.sf(T,df=v)
print(pvalue)
print()

alpha=0.1
if(pvalue> 1-alpha):
    print("Com nosso nível de significância alpha= 10%, nosso teste T REJEITA a hipótese inicial")
else:
    print("Com nosso nível de significância alpha= 10%, nosso teste T NÃO REJEITA a hipótese inicial")
    
alpha=0.05
if(pvalue> 1-alpha):
    print("Com nosso nível de significância alpha= 5%, nosso teste T REJEITA a hipótese inicial")
else:
    print("Com nosso nível de significância alpha= 5%, nosso teste T NÃO REJEITA a hipótese inicial")
    
alpha=0.01
if(pvalue> 1-alpha):
    print("Com nosso nível de significância alpha= 1%, nosso teste T REJEITA a hipótese inicial")
else:
    print("Com nosso nível de significância alpha= 1%, nosso teste T NÃO REJEITA a hipótese inicial")

0.964135517461

Com nosso nível de significância alpha= 10%, nosso teste T REJEITA a hipótese inicial
Com nosso nível de significância alpha= 5%, nosso teste T REJEITA a hipótese inicial
Com nosso nível de significância alpha= 1%, nosso teste T NÃO REJEITA a hipótese inicial


### Conclusão: ###

A validade do nosso teste T depende do nosso nível de confiança alpha em que trabalhamos, sendo alpha a probabilidade de que nosso teste não ser confiável. Não é possível, no entanto, assumir um nivel total de confiabilidade (alpha de 0%). Isto apenas seria possível caso existisse um espaço amostral que representasse com absoluta certeza a população total. 
  	
Queremos idealmente estudar nosso teste dentro de níveis altos de confiabilidade para uma análise final mais significativa. A mudança do valor de alpha pode confirmar ou rejeitar a hipotese nula, portanto decidimos fazer o teste com os tres valores mais comuns de alpha: 1%, 5% e 10%.
  	
Após os teste, percebemos que a hipotese nula é rejeitada com um nivel de significancia de 10% e 5%, indicando que há uma diferença consideravel na media do tempo nescessario para executar a tarefa do site em suas diferentes versões, e
mostrando que a posição dos links dos projetos influenciou nesse tempo. Porém, quando diminuimos o alpha (nivel de significancia) para 1%, a hipotese nula passou a não ser rejeitada, indicando que não há diferença entre as duas versões, em quesitos de tempo gasto para achar o projeto alvo. Isto se deve ao fato de nem todos os visitantes do site serem estranhos aos projetos de engenharia. 
  	
Concluímos que, ao analisar os dados coletados, com nível de confiabilidade de 95%, a nossa alteração na disposição dos itens do site beneficiou significativamente a navegabilidado, facilitando para os usuários encontrar as informações que procuram. 
   